# Parser construction example

This file demonstrates the process of constructing a parser file using `animals.csv` as a source dataset.

Before you start: `autoparser` requires an OpenAI API key to function. You should add yours to your environment, as described [here](https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety). 
Edit the `API_KEY` line below to match the name you gave yours.

In [ ]:
import autoparser
import pandas as pd
import os
API_KEY = os.environ.get("OPENAI_API_KEY")

In [ ]:
data = pd.read_csv("tests/sources/animal_data.csv")
data.head()

Let's generate a basic data dictionary from this data set. We want to use the configuration file set up for this dataset, located in the `tests` directory.

In [ ]:
writer = autoparser.DictWriter("tests/test_config.toml")
data_dict = writer.create_dict(data)
data_dict.head()

The 'Common Values' column indicates fields where there are a limited number of unique values, suggesting mapping to a controlled terminology may have been done, or might be required in the parser. The list of common values is every unique value in the field.

Notice that the Description column is empty. To proceed to the next step of the parser generation process, creating the mapping file linking source -> schema fields, this column must be filled. You can either do this by hand (the descriptions MUST be in english), or use autoparser's LLM functionality to do it for you, demonstrated below.

In [ ]:
dd_described = writer.generate_descriptions("fr", data_dict, key=API_KEY)
dd_described.head()

Now that we have a data dictionary with descriptions added, we can proceed to creating an intermediate mapping file:

In [ ]:
mapper = autoparser.Mapper("tests/schemas/animals.schema.json", dd_described, "fr", api_key=API_KEY, config="tests/test_config.toml")
mapping_dict = mapper.create_mapping(file_name='example_mapping.csv')

mapping_dict.head()

At this point, you should inspect the mapping file and look for fields/values that have been incorrectly mapped, and edit them where necessary.
The mapping file has been written out to [example_mapping.csv](example_mapping.csv). A good example is the 'loc_admin_1' field; the LLM often maps the common values provided to 'None' as the schema denotes this as a free-text field. Instead, delete these mapped values and the parsed data will contain the original free text.
Also note the warning above; the LLM should not have found fields to map to the 'country_iso3' or 'owner' fields. If the original data did contain an appropriate field for these, you should edit the mapping file accordingly.

Once you have edited the mapping file to your satisfaction, we can go ahead and create the TOML parser file, `example_parser.toml`:

In [ ]:
writer = autoparser.ParserGenerator("example_mapping.csv", "tests/schemas", "example", config="tests/test_config.toml")
writer.create_parser("example_parser.toml")

You can veiw/edit the created parser at [example_parser.toml](example_parser.toml), and try it out using [ADTL](https://github.com/globaldothealth/adtl).